In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.api.training import get_training_results
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

[2020-05-23 18:21:28,906] INFO in __init__: RobotEHR API


In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=18)

In [5]:
feature_pipeline = FeaturePipeline.load(id=22)

In [6]:
import morpher.config
from morpher.jobs import *

Using TensorFlow backend.


In [7]:
regexes = [
    'any__Diagnosis__',
    'any__Procedure__', 
    'any__Drug__', 
    'any__Material__', 
    'any__Encounter__', 
    'any__AlcoholUse__', 
    'any__DrugUse__', 
    'any__TobaccoUse__',
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [8]:
from itertools import product

In [9]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [10]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['other_viral_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.BORDERLINE, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [11]:
pipeline = training.execute(
    comment='baseline-5 ovi time series without RFE',
    version='35.1.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric_time_series",
    rfe__run=False,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7fcdc0034160>



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fetching data for Patient (...)


Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.


...

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.138ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 36.591ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 57.982ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 44.031ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 56.389ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 59.942ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 47.372ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 23.952ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 43.461ms

*** Tra

Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


*** Training of classifier ready. Time elapsed: 271.55ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 272.976ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 278.777ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 264.804ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 260.849ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 130.622ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 129.374ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 139.123ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 1

*** Training of classifier ready. Time elapsed: 21.129ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 16.566ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.332ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 20.45ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 11.487ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 9.21ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 9.058ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 11.937ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 11.693ms

*** Training of model 'LogisticRegression' started.
*** Training o

*** Training of classifier ready. Time elapsed: 379.802ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 388.158ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 406.149ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.862ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.254ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.259ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.049ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.074ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 1.844ms

*** Training of model 'DecisionTr

In [12]:
pipeline.id

214